In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
#importing important libraries
import cv2
from google.colab.patches import cv2_imshow
import dlib
import numpy as np
from PIL import Image
import os

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import dlib

#This function uses DLIB to get the boundary coordinates of face, eyes, etc.
def detect_forehead(image_path):
    # Load the pre-trained face detector and landmark predictor
    face_detector = dlib.get_frontal_face_detector()
    landmark_predictor = dlib.shape_predictor("/content/drive/MyDrive/DLIB FIles/shape_predictor_68_face_landmarks.dat") # Download the model from: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

    # Load the input image
    image = cv2.imread(image_path)

    # Convert the image to grayscale (dlib works with grayscale images)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_detector(gray_image)

    # Iterate over the detected faces
    for face in faces:

        # # Detect facial landmarks for the current face
        landmarks = landmark_predictor(gray_image, face)

        #UNCOMMENT THIS FOR LOOP TO OBTAIN THE FACIAL LANDMARKS:
        #for i, point in enumerate(landmarks.parts()):
        #  x, y = point.x, point.y
        #  cv2.putText(image, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)
        #  cv2.circle(image, (x, y), 1, (0, 255, 0), -1)

        # For simplicity, we assume that the forehead region is a rectangle above the eyes
    L = [image, landmarks]
    return L

In [ ]:
#Using the boundary coordinates obtained from detect_forehead() function, we are obtaining the
#top left and bottom right coordinates of forehead, eye, nose, mouth and other region.
#The other region is a superset of these above mentioned regions and covers the whole face.
#We are obtaining it for both human and humanoid images.
def humanROI(image, landmarks):
  #### FOREHEAD REGION:
  forehead_top_left = (landmarks.part(17).x, landmarks.part(17).y-80)
  forehead_bottom_right = (landmarks.part(26).x, landmarks.part(26).y-15)
  cv2.rectangle(image, forehead_top_left, forehead_bottom_right, (0, 255, 0), 1)

  ##### EYE REGION:
  eye_top_left = (landmarks.part(18).x-10, landmarks.part(26).y-15)
  eye_bottom_right = (landmarks.part(25).x+10, landmarks.part(29).y-5)
  cv2.rectangle(image, eye_top_left, eye_bottom_right, (0, 255, 0), 1)


  ###### NOSE REGION:
  nose_top_left = (landmarks.part(31).x-10, landmarks.part(29).y-5)
  nose_bottom_right = (landmarks.part(35).x+10, landmarks.part(34).y)
  cv2.rectangle(image, nose_top_left, nose_bottom_right, (0, 255, 0), 1)

  ####### LIPS REGION:
  lips_top_left = (landmarks.part(48).x, landmarks.part(34).y)
  lips_bottom_right = (landmarks.part(54).x, landmarks.part(57).y+10)
  cv2.rectangle(image, lips_top_left, lips_bottom_right, (0, 255, 0), 1)

  other_coordinates_top_left = (landmarks.part(0).x, landmarks.part(17).y-80)
  other_coordinates_bottom_right = (landmarks.part(16).x, landmarks.part(8).y)
  cv2.rectangle(image, other_coordinates_top_left, other_coordinates_bottom_right, (0, 255, 0), 1)

  return [[forehead_top_left, forehead_bottom_right], [eye_top_left, eye_bottom_right], [nose_top_left, nose_bottom_right], [lips_top_left, lips_bottom_right], [other_coordinates_top_left, other_coordinates_bottom_right]]

def humanoidROI(image, landmarks):
  ### FOR ROBOT:

  # #### FOREHEAD REGION:
  forehead_top_left = (landmarks.part(17).x, landmarks.part(17).y-130)
  forehead_bottom_right = (landmarks.part(26).x, landmarks.part(26).y-15)
  cv2.rectangle(image, forehead_top_left, forehead_bottom_right, (0, 255, 0), 1)


  # ##### EYE REGION:
  eye_top_left = (landmarks.part(18).x-10, landmarks.part(26).y-15)
  eye_bottom_right = (landmarks.part(25).x+10, landmarks.part(29).y-5)
  cv2.rectangle(image, eye_top_left, eye_bottom_right, (0, 255, 0), 1)


  # ###### NOSE REGION:
  nose_top_left = (landmarks.part(31).x-10, landmarks.part(29).y-5)
  nose_bottom_right = (landmarks.part(35).x+10, landmarks.part(34).y)
  cv2.rectangle(image, nose_top_left, nose_bottom_right, (0, 255, 0), 1)

  # ####### LIPS REGION:
  lips_top_left = (landmarks.part(48).x, landmarks.part(34).y)
  lips_bottom_right = (landmarks.part(54).x, landmarks.part(57).y+10)
  cv2.rectangle(image, lips_top_left, lips_bottom_right, (0, 255, 0), 1)

  other_coordinates_top_left = (landmarks.part(0).x, landmarks.part(17).y-130)
  other_coordinates_bottom_right = (landmarks.part(16).x, landmarks.part(8).y)
  cv2.rectangle(image, other_coordinates_top_left, other_coordinates_bottom_right, (0, 255, 0), 1)

  return [[forehead_top_left, forehead_bottom_right], [eye_top_left, eye_bottom_right], [nose_top_left, nose_bottom_right], [lips_top_left, lips_bottom_right], [other_coordinates_top_left, other_coordinates_bottom_right]]

In [ ]:
#For human
#In this function we're first obtaining the top left and bottom right coordinates of forehead, eyes, nose, mouth and other region for BSF images
#After that we're replicating the same for the corresponding low spatial and high spatial frequency images.

import os
import numpy as np
from PIL import Image

folder_path = '/content/drive/MyDrive/Filtered_Human_Images'
files = os.listdir(folder_path)
unfilteredImgs = []
unfilteredLandmarks = []
unfilteredCoords = []
unfilteredImageMatrix = []
imageMatrix = []
for file_name in files:
  full_file_name = folder_path + "/" + file_name
  if("low" not in full_file_name):
    unfilteredImgs.append(file_name)
    outpt = detect_forehead(full_file_name)
    image = outpt[0]
    landmarks = outpt[1]
    unfilteredLandmarks.append(landmarks)
    coords = humanROI(image, landmarks)
    unfilteredCoords.append(coords)
    unfilteredImageMatrix.append(image)

c = 0
ImgsNameList = []
ImgsLandmarks = []
ImgsCoords = []
for file_name in files:
  if(file_name not in unfilteredImgs):
    ImgsNameList.append(file_name)
    for j in range(len(unfilteredImgs)):
      if("matched-8" in file_name):
        if(file_name[18:28] in unfilteredImgs[j]):
          c = c+1
          image = cv2.imread(folder_path + "/" + file_name)
          landmarks = unfilteredLandmarks[j]
          ImgsLandmarks.append(unfilteredLandmarks[j])
          coords = humanROI(image, landmarks)
          ImgsCoords.append(coords)
          #cv2_imshow(image)
          imageMatrix.append(image)
      else:
        if(file_name[19:29] in unfilteredImgs[j]):
          c = c+1
          image = cv2.imread(folder_path + "/" + file_name)
          landmarks = unfilteredLandmarks[j]
          ImgsLandmarks.append(unfilteredLandmarks[j])
          coords = humanROI(image, landmarks)
          ImgsCoords.append(coords)
          #cv2_imshow(image)
          imageMatrix.append(image)

for i in range(len(unfilteredImgs)):
  image = cv2.imread(folder_path + "/" + unfilteredImgs[i])
  ImgsNameList.append(unfilteredImgs[i])
  ImgsLandmarks.append(unfilteredLandmarks[i])
  ImgsCoords.append(unfilteredCoords[i])
  imageMatrix.append(unfilteredImageMatrix[i])

for i in range(len(ImgsNameList)):
  destination_path = os.path.join("/content/drive/MyDrive/ROI_images_human", ImgsNameList[i])
  image = Image.fromarray(imageMatrix[i])
  image.save(destination_path, format="JPEG")


In [ ]:
#Here we're generating a CSV file containing the top left and bottom right coordinates of each region for each image.
import pandas as pd
df = pd.DataFrame({'file_name': ImgsNameList, 'coords': ImgsCoords})

destination_folder = "/content/drive/MyDrive"
csv_filename = "humanROIData.csv"
csv_path = f"{destination_folder}/{csv_filename}"
df.to_csv(csv_path, index=False)

In [ ]:
#For humanoid
#In this function we're first obtaining the top left and bottom right coordinates of forehead, eyes, nose, mouth and other region for BSF images
#After that we're replicating the same for the corresponding low spatial and high spatial frequency images.

import os
import numpy as np
from PIL import Image

folder_path = '/content/drive/MyDrive/Filtered_Robotic_Images'
files = os.listdir(folder_path)
unfilteredImgs = []
unfilteredLandmarks = []
unfilteredCoords = []
unfilteredImageMatrix = []
imageMatrix = []
for file_name in files:
  full_file_name = folder_path + "/" + file_name
  if("low" not in full_file_name):
    unfilteredImgs.append(file_name)
    outpt = detect_forehead(full_file_name)
    image = outpt[0]
    landmarks = outpt[1]
    unfilteredLandmarks.append(landmarks)
    coords = humanoidROI(image, landmarks)
    unfilteredCoords.append(coords)
    unfilteredImageMatrix.append(image)

print("l", len(unfilteredImgs))
print(unfilteredImgs)

c = 0
ImgsNameList = []
ImgsLandmarks = []
ImgsCoords = []
for file_name in files:
  if(file_name not in unfilteredImgs):
    ImgsNameList.append(file_name)
    for j in range(len(unfilteredImgs)):
      if("matched-8" in file_name):
        if(file_name[18:] in unfilteredImgs[j]):
          c = c+1
          image = cv2.imread(folder_path + "/" + file_name)
          landmarks = unfilteredLandmarks[j]
          ImgsLandmarks.append(unfilteredLandmarks[j])
          coords = humanoidROI(image, landmarks)
          ImgsCoords.append(coords)
          #cv2_imshow(image)
          imageMatrix.append(image)
      else:
        if(file_name[19:] in unfilteredImgs[j]):
          c = c+1
          image = cv2.imread(folder_path + "/" + file_name)
          landmarks = unfilteredLandmarks[j]
          ImgsLandmarks.append(unfilteredLandmarks[j])
          coords = humanoidROI(image, landmarks)
          ImgsCoords.append(coords)
          #cv2_imshow(image)
          imageMatrix.append(image)

print("c", c)
for i in range(len(unfilteredImgs)):
  image = cv2.imread(folder_path + "/" + unfilteredImgs[i])
  ImgsNameList.append(unfilteredImgs[i])
  ImgsLandmarks.append(unfilteredLandmarks[i])
  ImgsCoords.append(unfilteredCoords[i])
  imageMatrix.append(unfilteredImageMatrix[i])

print(len(ImgsNameList))
print(len(imageMatrix))


for i in range(len(ImgsNameList)):
  destination_path = os.path.join("/content/drive/MyDrive/ROI_images_humanoid", ImgsNameList[i])
  image = Image.fromarray(imageMatrix[i])
  image.save(destination_path, format="JPEG")


In [ ]:
#Humanoid
#For better ROI. We manually created ROIs for forehead, eyes, nose, mouth and other region for BSF humanoid images.
#Here we're loading those coordinates into a dataframe.
import pandas as pd
import ast

humanoidCoords = pd.read_csv('/content/drive/MyDrive/Humanoid Coordinates - Sheet1.csv')

unfilteredImgs = []
unfilteredCoords = []

column_names = humanoidCoords.columns

for j in range(1, humanoidCoords.shape[1]):
  unfilteredImgs.append(column_names[j])
  L = []
  for i in range(0, humanoidCoords.shape[0], 2):
    top_left = ast.literal_eval(humanoidCoords.iloc[i, j])
    bottom_right = ast.literal_eval(humanoidCoords.iloc[i+1, j])
    L.append([top_left, bottom_right])
  unfilteredCoords.append(L)

In [ ]:
#After loading the coordinates above for unfiltered images we're replicating the same for the corresponding low spatial and high spatial frequency images.
ImgsNameList = []
ImgsLandmarks = []
ImgsCoords = []
imageMatrix = []

for file_name in files:
  if(file_name not in unfilteredImgs):
    ImgsNameList.append(file_name)
    for j in range(len(unfilteredImgs)):
      if("matched-8" in file_name):
        if(file_name[18:] in unfilteredImgs[j]):
          c = c+1
          image = cv2.imread(folder_path + "/" + file_name)
          coords = unfilteredCoords[j]
          ImgsCoords.append(coords)
          #cv2_imshow(image)
          imageMatrix.append(image)
      else:
        if(file_name[19:] in unfilteredImgs[j]):
          c = c+1
          image = cv2.imread(folder_path + "/" + file_name)
          coords = unfilteredCoords[j]
          ImgsCoords.append(coords)
          #cv2_imshow(image)
          imageMatrix.append(image)

for i in range(len(unfilteredImgs)):
  image = cv2.imread(folder_path + "/" + unfilteredImgs[i])
  ImgsNameList.append(unfilteredImgs[i])
  ImgsCoords.append(unfilteredCoords[i])
  imageMatrix.append(image)

for i in range(len(ImgsNameList)):
  coords = ImgsCoords[i]
  for j in range(len(coords)):
    top_left = coords[j][0]
    bottom_right = coords[j][1]
    cv2.rectangle(imageMatrix[i], top_left, bottom_right, (0, 255, 0), 1)

for i in range(len(ImgsNameList)):
  destination_path = os.path.join("/content/drive/MyDrive/ROI_images_humanoid_2", ImgsNameList[i])
  image = Image.fromarray(imageMatrix[i])
  image.save(destination_path, format="JPEG")

In [ ]:
#Here we're generating a CSV file containing the top left and bottom right coordinates of each region for each image.
import pandas as pd
df = pd.DataFrame({'file_name': ImgsNameList, 'coords': ImgsCoords})

destination_folder = "/content/drive/MyDrive"
csv_filename = "humanoidROIData.csv"
csv_path = f"{destination_folder}/{csv_filename}"
df.to_csv(csv_path, index=False)

In [ ]:
#Robotic
#For better ROI. We manually created ROIs for eyes and mouth regions for BSF robotic images.
#Here we're loading those coordinates into a dataframe.
import pandas as pd
import ast

robotCoords = pd.read_csv('/content/drive/MyDrive/Robot Coordinates - Sheet1.csv')

unfilteredImgs = []
unfilteredCoords = []

column_names = robotCoords.columns

for j in range(1, robotCoords.shape[1]):
  unfilteredImgs.append(column_names[j])
  L = []
  for i in range(0, robotCoords.shape[0], 2):
    top_left = ast.literal_eval(robotCoords.iloc[i, j])
    bottom_right = ast.literal_eval(robotCoords.iloc[i+1, j])
    L.append([top_left, bottom_right])
  unfilteredCoords.append(L)

print(unfilteredImgs)
print(unfilteredCoords)

['matched-c10.jpg', 'matched-c11.jpg', 'matched-c12.jpg', 'matched-c14.jpg', 'matched-c16.jpg', 'matched-c17.jpg', 'matched-c27.jpg', 'matched-c33.jpg', 'matched-c35.jpg', 'matched-c36.jpg', 'matched-c37.jpg', 'matched-c38.jpg', 'matched-c41.jpg', 'matched-c42.jpg', 'matched-c43.jpg']
[[[(771, 318), (1176, 619)], [(868, 619), (1118, 711)]], [[(788, 375), (1126, 619)], [(839, 619), (1104, 774)]], [[(785, 492), (1142, 681)], [(802, 681), (1139, 770)]], [[(726, 332), (1148, 601)], [(776, 601), (1076, 781)]], [[(712, 411), (1210, 587)], [(760, 587), (1182, 729)]], [[(749, 516), (1184, 684)], [(790, 684), (1154, 789)]], [[(769, 352), (1181, 592)], [(772, 592), (1184, 770)]], [[(772, 442), (1158, 574)], [(869, 574), (1079, 711)]], [[(732, 527), (1144, 709)], [(845, 709), (1046, 789)]], [[(825, 449), (1115, 603)], [(848, 603), (1088, 766)]], [[(741, 458), (1150, 684)], [(832, 684), (1052, 775)]], [[(744, 338), (1211, 550)], [(859, 550), (1094, 725)]], [[(752, 384), (1170, 581)], [(830, 581), 

In [ ]:
#After loading the coordinates above for BSF images we're replicating the same for the corresponding low spatial and high spatial frequency images.

ImgsNameList = []
ImgsLandmarks = []
ImgsCoords = []
imageMatrix = []

folder_path = '/content/drive/MyDrive/screenshots_ROBOTS'
files = os.listdir(folder_path)

c = 0
for file_name in files:
  if(file_name not in unfilteredImgs):
    ImgsNameList.append(file_name)
    for j in range(len(unfilteredImgs)):
      if("matched-8" in file_name):
        if(file_name[18:] in unfilteredImgs[j]):
          image = cv2.imread(folder_path + "/" + file_name)
          coords = unfilteredCoords[j]
          ImgsCoords.append(coords)
          #cv2_imshow(image)
          imageMatrix.append(image)
          c = c+1
      else:
        if(file_name[19:] in unfilteredImgs[j]):
          image = cv2.imread(folder_path + "/" + file_name)
          coords = unfilteredCoords[j]
          ImgsCoords.append(coords)
          #cv2_imshow(image)
          imageMatrix.append(image)
          c = c+1

for i in range(len(unfilteredImgs)):
  image = cv2.imread(folder_path + "/" + unfilteredImgs[i])
  ImgsNameList.append(unfilteredImgs[i])
  ImgsCoords.append(unfilteredCoords[i])
  imageMatrix.append(image)

for i in range(len(ImgsNameList)):
  coords = ImgsCoords[i]
  for j in range(len(coords)):
    top_left = coords[j][0]
    bottom_right = coords[j][1]
    cv2.rectangle(imageMatrix[i], top_left, bottom_right, (0, 255, 0), 1)

for i in range(len(ImgsNameList)):
  destination_path = os.path.join("/content/drive/MyDrive/ROI_images_robot", ImgsNameList[i])
  image = Image.fromarray(imageMatrix[i])
  image.save(destination_path, format="JPEG")


In [ ]:
#Here we're generating a CSV file containing the top left and bottom right coordinates of each region for each image.
import pandas as pd
df = pd.DataFrame({'file_name': ImgsNameList, 'coords': ImgsCoords})

destination_folder = "/content/drive/MyDrive"
csv_filename = "robotROIData.csv"
csv_path = f"{destination_folder}/{csv_filename}"
df.to_csv(csv_path, index=False)